In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        if (filename.endswith(".csv")) or (filename.endswith(".pt")):
            print(os.path.join(dirname, filename))

Due to data sharing policy of the original dataset owners, some outputs are ommitted.

In [ ]:
train_df=pd.read_csv("dataset/Tamil/train-20250127T110108Z-001/train/train.csv")
train_df.head()

In [ ]:
val_df=pd.read_csv("dataset/Tamil/dev-20250127T110118Z-001/dev/dev.csv")
val_df.head()

In [ ]:
test_df=pd.read_csv("dataset/Tamil/test-20250127T110123Z-001/test/test.csv")
test_df.head()

In [5]:
import os
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import AutoModel, AutoTokenizer, AutoProcessor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm

c:\Users\USER\anaconda3\envs\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Paths
TRAIN_IMAGE_FOLDER = "dataset/Tamil/train-20250127T110108Z-001/train/"
VAL_IMAGE_FOLDER = "dataset/Tamil/dev-20250127T110118Z-001/dev/"
TEST_IMAGE_FOLDER = "dataset/Tamil/test-20250127T110123Z-001/test/"

In [7]:
# Parameters
IMAGE_MODEL_NAME = "zer0int/CLIP-GmP-ViT-L-14"  # Example vision model
TEXT_MODEL_NAME = "PosteriorAI/dravida_llama2_7b"  # Example Tamil BERT model
BATCH_SIZE = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def load_embeddings(embedding_path):
    if os.path.exists(embedding_path):
        print(f"Loading embeddings from {embedding_path}")
        return torch.load(embedding_path)
    else:
        raise FileNotFoundError(f"Embeddings file not found at {embedding_path}")

In [9]:
# Load training, validation, and test embeddings
train_image_embeddings = load_embeddings("train_image_embeddings.pt")
val_image_embeddings = load_embeddings("val_image_embeddings.pt")
test_image_embeddings = load_embeddings("test_image_embeddings.pt")

train_text_embeddings = load_embeddings("train_text_embeddings.pt")
val_text_embeddings = load_embeddings("val_text_embeddings.pt")
test_text_embeddings = load_embeddings("test_text_embeddings.pt")

Loading embeddings from train_image_embeddings.pt
Loading embeddings from val_image_embeddings.pt
Loading embeddings from test_image_embeddings.pt
Loading embeddings from train_text_embeddings.pt
Loading embeddings from val_text_embeddings.pt
Loading embeddings from test_text_embeddings.pt


In [10]:
def combine_embeddings(image_embeddings, text_embeddings, df, has_labels=True):
    combined_embeddings = []
    labels = [] if has_labels else None

    for idx, row in df.iterrows():
        image_id = row["image_id"]
        if image_id in image_embeddings and image_id in text_embeddings:
            # Squeeze to remove unnecessary dimensions
            image_embedding = image_embeddings[image_id].squeeze()
            text_embedding = text_embeddings[image_id].squeeze()

            # Combine image and text embeddings
            combined = torch.cat([image_embedding, text_embedding], dim=-1)
            combined_embeddings.append(combined)

            if has_labels:
                labels.append(row["labels"])

    if has_labels:
        return torch.stack(combined_embeddings), torch.tensor(labels)
    else:
        return torch.stack(combined_embeddings)

In [11]:
X_train, y_train = combine_embeddings(train_image_embeddings, train_text_embeddings, train_df)
X_val, y_val = combine_embeddings(val_image_embeddings, val_text_embeddings, val_df)
X_test = combine_embeddings(test_image_embeddings, test_text_embeddings, test_df, has_labels=False)

print(f"Training data shape: {X_train.shape}, Labels: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, Labels: {y_val.shape}")
print(f"Test data shape: {X_test.shape}")


Training data shape: torch.Size([416, 4864]), Labels: torch.Size([416])
Validation data shape: torch.Size([15, 4864]), Labels: torch.Size([15])
Test data shape: torch.Size([43, 4864])


In [12]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# Define the MLP model
class MLPClassifier(nn.Module):
    def _init_(self, input_size, hidden_sizes, output_size):
        super(MLPClassifier, self)._init_()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_sizes[1], output_size),
            nn.Sigmoid(),  # Use Sigmoid for binary classification
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_sizes = [1024, 512]
output_size = 1
batch_size = 32
num_epochs = 10
learning_rate = 0.0005

In [15]:
# Prepare datasets and loaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Initialize model, loss function, and optimizer
model = MLPClassifier(input_size, hidden_sizes, output_size).to(device)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [17]:
import pandas as pd

def train_and_save_best_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, save_dir):
    best_f1 = -float('inf')  # Initialize to a very low value to track the best model
    best_model_path = None  # Path to save the best model

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        all_train_preds, all_train_labels = [], []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            outputs = model(inputs).squeeze()

            # Compute loss
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # Collect predictions and labels for metrics
            preds = (outputs > 0.5).int()
            all_train_preds.extend(preds.tolist())
            all_train_labels.extend(labels.tolist())

        # Calculate training metrics
        train_accuracy = accuracy_score(all_train_labels, all_train_preds)
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(
            all_train_labels, all_train_preds, average="macro"
        )

        # Validation phase
        model.eval()
        val_loss = 0
        all_val_preds, all_val_labels = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float()
                outputs = model(inputs).squeeze()

                # Compute loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Collect predictions and labels for metrics
                preds = (outputs > 0.5).int()
                all_val_preds.extend(preds.tolist())
                all_val_labels.extend(labels.tolist())

        # Calculate validation metrics
        val_accuracy = accuracy_score(all_val_labels, all_val_preds)
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(
            all_val_labels, all_val_preds, average="macro",
        )

        # Print metrics for the current epoch
        print(
            f"Epoch {epoch + 1}/{num_epochs}: "
            f"Train Loss: {train_loss / len(train_loader):.4f}, "
            f"Train Acc: {train_accuracy:.4f}, Prec: {train_precision:.4f}, Rec: {train_recall:.4f}, F1: {train_f1:.4f} | "
            f"Val Loss: {val_loss / len(val_loader):.4f}, "
            f"Val Acc: {val_accuracy:.4f}, Prec: {val_precision:.4f}, Rec: {val_recall:.4f}, F1: {val_f1:.4f}"
        )

        # Save the model if it has the best F1 score on validation
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_path = f"{save_dir}/best_model_epoch_{epoch + 1}_f1_{val_f1:.4f}.pth"
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with F1: {val_f1:.4f} at epoch {epoch + 1}")

    return best_model_path

In [ ]:
import os

save_dir = "./saved_models"
os.makedirs(save_dir, exist_ok=True)


best_model_path = train_and_save_best_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    save_dir=save_dir
)

In [19]:
def predict_and_generate_submission(test_loader, best_model_path, submission_file_path):
    # Load the best model with weights_only=True to avoid security warnings
    model = MLPClassifier(input_size, hidden_sizes, output_size).to(device)
    model.load_state_dict(torch.load(best_model_path, weights_only=True))
    model.eval()  # Set the model to evaluation mode

    test_predictions = []
    with torch.no_grad():
        for inputs in test_loader:
            # Ensure inputs are converted to a tensor and stacked into a batch if necessary
            if isinstance(inputs, list):
                # Convert each item to tensor using .detach() to avoid the user warning
                inputs = [i.clone().detach().to(device) if isinstance(i, torch.Tensor) else torch.tensor(i).to(device) for i in inputs]
                inputs = torch.stack(inputs)  # Stack them into a batch tensor
            else:
                inputs = inputs.to(device)  # If inputs is already a tensor, move it to device

            outputs = model(inputs).squeeze()

            # Predict binary labels
            preds = (outputs > 0.5).int()
            test_predictions.extend(preds.tolist())

    # Prepare the submission DataFrame
    submission_df = pd.DataFrame({
        'id': [i for i in test_df['image_id']],
        'predictions': test_predictions
    })

    # Save the predictions to a CSV file
    submission_df.to_csv(submission_file_path, index=False)
    print(f"Submission file saved to {submission_file_path}")
    
    return submission_df

In [ ]:
submission_file_path = "submission.csv"
submission_df = predict_and_generate_submission(test_loader=test_loader, best_model_path=best_model_path, submission_file_path=submission_file_path)

In [ ]:
submission_df.head()